In [1]:
import opensim as osim
import os
import xml.etree.ElementTree as ET
import re

# IMPORTANT DO NOT REMOVE
osim.ModelVisualizer.addDirToGeometrySearchPaths("C:/OpenSim 4.5/Geometry")

action = "Shallow Squat"
model_name = "Hanif-Scaled.osim"

base_path = f"C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/{action}"
model_path = "C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Models"
setup_path = "C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Scripts/Setup_XMLs"
setup_path_relative = "../../../../Scripts/Setup_XMLs"

ik_path = os.path.join(base_path, 'IK').replace("\\", "/")
grf_path = os.path.join(base_path, 'ID', 'GRF').replace("\\", "/")
rra_path = os.path.join(base_path, 'RRA').replace("\\", "/")

model_file = os.path.join(model_path, model_name).replace("\\", "/")
setup_xml = os.path.join(setup_path, "rra_setup.xml").replace("\\", "/")
task_xml = os.path.join(setup_path_relative, "Boiler_XMLs", "rra_tasks.xml").replace("\\", "/")
actuator_xml = os.path.join(setup_path_relative, "Boiler_XMLs", "rra_actuators.xml").replace("\\", "/")  

ik_files = [f for f in os.listdir(ik_path) if f.endswith('.mot')]
grf_xmls = [f for f in os.listdir(grf_path) if f.endswith('.xml')]

print(f"Loading model from:", model_file)
print(f"IK files found {len(ik_files)}:", ik_files)
print(f"GRF files found {len(grf_xmls)}:", grf_xmls)

Loading model from: C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Models/Hanif-Scaled.osim
IK files found 9: ['1_Hanif_Shallow_2_5_patched.mot', '1_Hanif_Shallow_5_0_patched.mot', '1_Hanif_Shallow_7_5_patched.mot', '2_Hanif_Shallow_2_5_patched.mot', '2_Hanif_Shallow_5_0_patched.mot', '2_Hanif_Shallow_7_5_patched.mot', '3_Hanif_Shallow_2_5_patched.mot', '3_Hanif_Shallow_5_0_patched.mot', '3_Hanif_Shallow_7_5_patched.mot']
GRF files found 9: ['1_Hanif_Shallow_2_5_grf.xml', '1_Hanif_Shallow_5_0_grf.xml', '1_Hanif_Shallow_7_5_grf.xml', '2_Hanif_Shallow_2_5_grf.xml', '2_Hanif_Shallow_5_0_grf.xml', '2_Hanif_Shallow_7_5_grf.xml', '3_Hanif_Shallow_2_5_grf.xml', '3_Hanif_Shallow_5_0_grf.xml', '3_Hanif_Shallow_7_5_grf.xml']


In [2]:
# Generate RRA Result Folders if it doesn't exist

for ik_file in ik_files:
    trial_name = ik_file.replace('.mot', '')
    rra_trial_path = os.path.join(rra_path, trial_name).replace("\\", "/")
    if not os.path.exists(rra_trial_path):
        os.makedirs(rra_trial_path)
        print(f"Created RRA result folder for {trial_name} at {rra_trial_path}")

Created RRA result folder for 1_Hanif_Shallow_2_5_patched at C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/1_Hanif_Shallow_2_5_patched
Created RRA result folder for 1_Hanif_Shallow_5_0_patched at C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/1_Hanif_Shallow_5_0_patched
Created RRA result folder for 1_Hanif_Shallow_7_5_patched at C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/1_Hanif_Shallow_7_5_patched
Created RRA result folder for 2_Hanif_Shallow_2_5_patched at C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/2_Hanif_Shallow_2_5_patched
Created RRA result folder for 2_Hanif_Shallow_5_0_patched at C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/2_Hanif_Shallow_5_0_patched


In [3]:
# Reduce Residuals Batch Processing


for ik_file, grf_xml in zip(ik_files, grf_xmls):
    
    # Core RRA Logic

    storage = osim.Storage(os.path.join(ik_path, ik_file).replace("\\", "/"))
    start_time = storage.getFirstTime()
    end_time = storage.getLastTime()
    print(f"IK File: {ik_file}, Start Time: {start_time}, End Time: {end_time}")

    results_folder = os.path.join(rra_path, ik_file.replace('.mot', '')).replace("\\", "/")

    osim.Logger.removeFileSink()
    osim.Logger.addFileSink(os.path.join(results_folder, "opensim.log"))

    # Copy actuators.xml & tasks.xml
    # shutil.copy2(task_xml, results_folder)
    # shutil.copy2(actuator_xml, results_folder)
  
    tree = ET.parse(setup_xml)

    for elem in tree.iter('model_file'):
        elem.text = model_file
    for elem in tree.iter('force_set_files'):
        elem.text = actuator_xml
    for elem in tree.iter('results_directory'):
        elem.text = results_folder
    for elem in tree.iter('initial_time'):
        elem.text = str(start_time)
    for elem in tree.iter('final_time'):
        elem.text = str(end_time)
    for elem in tree.iter('external_loads_file'):
        elem.text = os.path.join(grf_path, grf_xml).replace("\\", "/")
    for elem in tree.iter('desired_kinematics_file'):
        elem.text = os.path.join(ik_path, ik_file).replace("\\", "/")
    for elem in tree.iter('task_set_file'):
        elem.text = task_xml
    for elem in tree.iter('output_model_file'):
        elem.text = os.path.join(results_folder, ik_file.replace(".mot", "_model_adjusted.osim")).replace("\\", "/")

    output_xml = ik_file.replace('.mot', '_setup.xml')
    tree.write(os.path.join(results_folder, output_xml), encoding="UTF-8", xml_declaration=True)
  
    print(f"Results will be saved in: {results_folder}")
    rra_tool = osim.RRATool(os.path.join(results_folder, output_xml).replace("\\", "/"))  
    rra_tool.run()

    # Torso mass adjustment (based on RRA recommendation)

    with open(os.path.join(results_folder, "opensim.log"), "r") as f:
        log = f.read()

    match = re.search(r"Total mass change:\s*([\-0-9\.Ee+]+)", log)

    if match:
        total_mass_change = float(match.group(1))
        print("Total mass change:", total_mass_change)
    else:
        print("New mass not found")
        continue

    adjusted_model = osim.Model(os.path.join(results_folder, ik_file.replace(".mot", "_model_adjusted.osim")).replace("\\", "/"))
    adjusted_model.initSystem()

    torso = adjusted_model.getBodySet().get("torso")
    old_mass = torso.getMass()
    new_mass = old_mass + total_mass_change

    print("Old torso mass:", old_mass)
    print("New torso mass:", new_mass)

    torso.setMass(new_mass)

    adjusted_model.printToXML(os.path.join(results_folder, ik_file.replace(".mot", "_model_adjusted.osim")).replace("\\", "/"))

    print("Saved mass-adjusted model!!")

IK File: 1_Hanif_Shallow_2_5_patched.mot, Start Time: 0.2, End Time: 1.7
Results will be saved in: C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/1_Hanif_Shallow_2_5_patched
Total mass change: -1.75463
Old torso mass: 25.894512975523064
New torso mass: 24.139882975523065
Saved mass-adjusted model!!
IK File: 1_Hanif_Shallow_5_0_patched.mot, Start Time: 0.2, End Time: 2.2
Results will be saved in: C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/1_Hanif_Shallow_5_0_patched
Total mass change: 0.713856
Old torso mass: 25.894512975523064
New torso mass: 26.608368975523064
Saved mass-adjusted model!!
IK File: 1_Hanif_Shallow_7_5_patched.mot, Start Time: 0.8, End Time: 3.1
Results will be saved in: C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Shallow Squat/RRA/1_Hanif_Shallow_7_5_patched
Total mass change: 3.34574
Old tor